In [ ]:
from loica import *
import matplotlib.pyplot as plt
import numpy as np

### Repressilator

In [ ]:
rep = GeneticNetwork()

laci = Regulator(name='LacI', degradation_rate=1, init_concentration=5)
tetr = Regulator(name='TetR', degradation_rate=1)
ci = Regulator(name='cI', degradation_rate=1)
rep.add_regulator(laci)
rep.add_regulator(tetr)
rep.add_regulator(ci)

sfp1 = Reporter(name='SFP1', degradation_rate=1)
rep.add_reporter(sfp1)
sfp2 = Reporter(name='SFP2', degradation_rate=1)
rep.add_reporter(sfp2)
sfp3 = Reporter(name='SFP3', degradation_rate=1)
rep.add_reporter(sfp3)

rep.add_tu(RepressedRepressor(input=ci, output=laci, a=100, b=0, K=1, n=2))
rep.add_tu(RepressedRepressor(input=laci, output=tetr, a=100, b=0, K=1, n=2))
rep.add_tu(RepressedRepressor(input=tetr, output=ci, a=100, b=0, K=1, n=2))

rep.add_tu(RepressedRepressor(input=ci, output=sfp1, a=100, b=0, K=1, n=2))
rep.add_tu(RepressedRepressor(input=laci, output=sfp2, a=100, b=0, K=1, n=2))
rep.add_tu(RepressedRepressor(input=tetr, output=sfp3, a=100, b=0, K=1, n=2))

In [ ]:
def growth_rate(t):
    return gompertz_growth_rate(t, 0.01, 1, 1, 4)

def biomass(t):
    return gompertz(t, 0.01, 1, 1, 4)

def profile(biomass, gr, t):
    return gr * 10 + 1
    
metab = SimulatedMetabolism(biomass, growth_rate, profile)
sample = Sample(circuit=rep, metabolism=metab)
assay = Assay([sample], 100, 0.25)
assay.run()

In [ ]:
m = assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='SFP1'].plot(x='Time', y='Measurement', ax=ax)
m[m.Signal=='SFP2'].plot(x='Time', y='Measurement', ax=ax)
m[m.Signal=='SFP3'].plot(x='Time', y='Measurement', ax=ax)

### Toggle switch

In [ ]:
from flapjack import *
fj = Flapjack(url_base='3.128.232.8:8000')
fj.log_in(username='timrudge', password='chicken')

In [ ]:
dna = fj.create('dna', name='Toggle')
vector = fj.create('vector', name='Toggle', dnas=dna.id)
tog = GeneticNetwork(vector=vector.id[0])

laci = Regulator(name='LacI', degradation_rate=1, init_concentration=0.1)
ci = Regulator(name='cI', degradation_rate=1)
tog.add_regulator(laci)
tog.add_regulator(ci)

cfp = fj.get('signal', name='CFP')
yfp = fj.get('signal', name='YFP')
sfp1 = Reporter(name='CFP', degradation_rate=1, signal_id=cfp.id[0])
tog.add_reporter(sfp1)
sfp2 = Reporter(name='YFP', degradation_rate=1, signal_id=yfp.id[0])
tog.add_reporter(sfp2)

tog.add_tu(RepressedRepressor(input=laci, output=ci, a=10, b=0, K=1, n=2))
tog.add_tu(RepressedRepressor(input=ci, output=laci, a=10, b=0, K=1, n=2))

tog.add_tu(RepressedRepressor(input=ci, output=sfp1, a=10, b=0, K=1, n=2))
tog.add_tu(RepressedRepressor(input=laci, output=sfp2, a=10, b=0, K=1, n=2))

In [ ]:
study = fj.get('study', name='Loica testing')
if len(study)==0:
    study = fj.create('study', name='Loica testing', description='Test')
media = fj.get('media', name='Loica')
if len(media)==0:
    media = fj.create('media', name='Loica', description='Simulated loica media')
strain = fj.get('strain', name='Loica strain')
if len(strain)==0:
    strain = fj.create('strain', name='Loica strain', description='Loica test strain')

biomass_signal = fj.get('signal', name='OD')
sample = Sample(circuit=tog, 
                metabolism=metab,
                media=media.id[0],
                strain=strain.id[0],
                vector=vector.id[0]
               )
assay = Assay([sample], 
              n_measurements=100, 
              interval=0.25,
              name='Loica toggle',
              description='Simulated toggle switch generated by loica',
              biomass_signal_id=biomass_signal.id[0]
             )
assay.run()

In [ ]:
m = assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='CFP'].plot(x='Time', y='Measurement', ax=ax)
m[m.Signal=='YFP'].plot(x='Time', y='Measurement', ax=ax)

### Upload simulated data to flapjack

In [ ]:
assay.upload(fj, study.id[0])